# OHCL - Analytics

### Import Library

In [61]:
import numpy as np
import pandas as pd
import numpy as np
import pandas_ta as ta
import seaborn as sns

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 120
import warnings
warnings.filterwarnings('ignore')

### Load Price Data

In [62]:
import os
from pathlib import Path
notebook_path = os.getcwd()
algo_dir = Path(notebook_path).parent.parent
csv_file = str(algo_dir) + '/vn-stock-data/VN30ps/VN30F1M_5minutes.csv'
is_file = os.path.isfile(csv_file)
if is_file:
    dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
else:
    print('remote')
    dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)

In [63]:
data = dataset.copy()

In [64]:
# data = data[(data.index > '2020-11-01 00:00:00') & (data.index < '2024-10-01 00:00:00')]
data = data[data.index > '2020-11-01 00:00:00']

In [65]:
data['close_14h15'] = data.Close
data['close_14h20'] = data.Close
data['close_14h25'] = data.Close

In [66]:
data

,Open,High,Low,Close,Volume,close_14h15,close_14h20,close_14h25
Date,,,,,,,,
2020-11-02 09:00:00,900.1,900.2,899.3,900.1,1910,900.1,900.1,900.1
2020-11-02 09:05:00,900.2,900.2,898.7,899.4,1670,899.4,899.4,899.4
2020-11-02 09:10:00,899.5,900.0,899.0,899.5,1329,899.5,899.5,899.5
2020-11-02 09:15:00,899.4,899.5,898.2,898.6,1722,898.6,898.6,898.6
2020-11-02 09:20:00,898.5,898.6,896.5,898.2,2939,898.2,898.2,898.2
...,...,...,...,...,...,...,...,...
2025-02-14 14:15:00,1343.0,1343.0,1340.3,1341.3,7141,1341.3,1341.3,1341.3
2025-02-14 14:20:00,1340.9,1341.9,1340.5,1341.4,4593,1341.4,1341.4,1341.4
2025-02-14 14:25:00,1341.1,1342.5,1340.7,1342.5,4207,1342.5,1342.5,1342.5


In [67]:
def cal_price_14h15(tick):
  tick = tick[100*tick.index.hour+tick.index.minute == 1415]
  if not tick.empty:
    return tick[0]
      
def cal_price_14h20(tick):
  tick = tick[100*tick.index.hour+tick.index.minute == 1420]
  if not tick.empty:
    return tick[0]
      
def cal_price_14h25(tick):
  tick = tick[100*tick.index.hour+tick.index.minute == 1425]
  if not tick.empty:
    return tick[0]

In [68]:
day_data = data.resample("D").agg({
    'Open': 'first',
    'Close': 'last',
    'High': 'max',
    'Low': 'min',
    'close_14h15': cal_price_14h15,
    'close_14h20': cal_price_14h20,
    'close_14h25': cal_price_14h25
    })

In [69]:
day_data.dropna(subset=['Close'], inplace=True)

In [70]:
day_data

,Open,Close,High,Low,close_14h15,close_14h20,close_14h25
Date,,,,,,,
2020-11-02,900.1,904.0,907.0,895.4,899.0,904.5,907.0
2020-11-03,909.3,908.3,910.8,903.7,909.7,909.0,908.5
2020-11-04,909.0,912.3,918.1,906.7,915.0,912.6,913.5
2020-11-05,913.7,910.2,919.9,910.0,914.2,914.4,910.8
2020-11-06,911.2,908.3,912.7,903.3,910.6,910.9,910.0
...,...,...,...,...,...,...,...
2025-02-10,1333.4,1330.5,1337.6,1325.2,1333.6,1332.2,1330.0
2025-02-11,1332.5,1334.1,1334.1,1327.6,1330.5,1331.6,1332.4
2025-02-12,1337.4,1329.5,1342.0,1329.5,1336.8,1335.5,1332.8


In [71]:
day_data['return'] = day_data.apply(lambda r: r['Close'] - r['close_14h25'], axis=1)

In [72]:
def cal_signal_v1(r):
    signal = ''
    if r['close_14h25'] > r['close_14h20'] > r['close_14h15']:
        signal = 'long'
    elif r['close_14h25'] < r['close_14h20'] < r['close_14h15']:
        signal = 'short'
    return signal

In [73]:
day_data['signal_v1'] = day_data.apply(lambda r: cal_signal_v1(r), axis=1)

In [74]:
day_data[day_data.signal_v1 != '']

,Open,Close,High,Low,close_14h15,close_14h20,close_14h25,return,signal_v1
Date,,,,,,,,,
2020-11-02,900.1,904.0,907.0,895.4,899.0,904.5,907.0,-3.0,long
2020-11-03,909.3,908.3,910.8,903.7,909.7,909.0,908.5,-0.2,short
2020-11-09,914.2,923.0,923.0,913.8,921.0,921.2,921.5,1.5,long
2020-11-11,922.6,921.4,922.8,912.2,913.5,919.2,922.1,-0.7,long
2020-11-12,921.4,927.9,927.9,920.8,927.3,927.0,926.5,1.4,short
...,...,...,...,...,...,...,...,...,...
2025-02-07,1338.5,1343.3,1347.4,1335.0,1344.2,1344.0,1343.3,0.0,short
2025-02-10,1333.4,1330.5,1337.6,1325.2,1333.6,1332.2,1330.0,0.5,short
2025-02-11,1332.5,1334.1,1334.1,1327.6,1330.5,1331.6,1332.4,1.7,long


In [75]:
day_data[day_data.signal_v1 == 'long']['return'].sum()

82.39999999999907

In [76]:
len(day_data[day_data.signal_v1 == 'long'])

308

In [77]:
day_data[day_data.signal_v1 == 'short']['return'].sum()

-149.90000000000077

In [78]:
len(day_data[day_data.signal_v1 == 'short'])

267